<a href="https://colab.research.google.com/github/PosgradoMNA/Proyecto_Integrador_EQ_2/blob/main/Avance_4_EQ2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [75]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, cross_val_score

In [76]:
from google.colab import drive
drive.mount('/content/drive')
import os
DIR = "/content/drive/MyDrive/Colab Notebooks/ProyectoIntegradorEQ2"
#DIR = "/content/drive/MyDrive/Colab Notebooks/MNA/Proyecto integrador/Data"
os.chdir(DIR)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [77]:
# Cargar dataset
df = pd.read_csv("Consultas.csv")
df.head()

,Consultas principales,Clics,Impresiones,CTR,Posición
0,precio de cortinas metálicas para negocio,34,1011,3.36%,4.67
1,scosnaya,33,74,44.59%,1.20
2,cuanto cuesta una cortina metálica para negocio,27,1129,2.39%,3.71
3,cortinas de acero precios,26,692,3.76%,5.73
4,precio de cortinas para negocio,20,631,3.17%,3.33


In [78]:
# 1.- Check de valores faltantes, incluyendo valores de mediana en los faltantes.
if df['Posición'].isnull().any():
    median_position = df['Posicion'].median()
    df['Posicion'].fillna(median_position, inplace=True)

In [79]:
# CTR conversión a float y quitando '%'.
df['CTR'] = df['CTR'].astype(str).str.rstrip('%').astype('float') / 100

In [80]:
# 2. Outliers:
q1 = df['Impresiones'].quantile(0.25)
q3 = df['Impresiones'].quantile(0.75)
iqr = q3 - q1
upper_bound = q3 + 1.5 * iqr
lower_bound = q1 - 1.5 * iqr

#quitando outliers
data = df[(df['Impresiones'] >= lower_bound) & (df['Impresiones'] <= upper_bound)]

In [81]:
#Preparando los valores de X e y - incluyendo OneHotEncoder para convertir consultas principales
X = df.drop('Clics', axis=1)
y = df['Clics']

ct = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['Impresiones', 'Posición', 'CTR']),
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['Consultas principales'])
    ],
    remainder='passthrough'
)


In [85]:
# Define los modelos a evaluar
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest': RandomForestRegressor(),
    'Gradient Boosting': GradientBoostingRegressor(),
    'Support Vector Regression': SVR()
}

# Evaluar los modelos con validación cruzada
def evaluate_model(model, X, y):
    cv_scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
    rmse_scores = np.sqrt(-cv_scores)
    return rmse_scores.mean()

# Entrenar y evaluar cada modelo
results = {}
for name, model in models.items():
    pipeline = Pipeline([
        ('transformer', ct),
        ('regressor', model)
    ])
    rmse = evaluate_model(pipeline, X, y)
    results[name] = rmse
    print(f"Model: {name}, Average RMSE: {rmse}")

# Seleccionamos los dos mejores
sorted_results = sorted(results.items(), key=lambda item: item[1])
top_2_models = sorted_results[:2]

print("\n Los 2 mejores Modelos son: ")
for name, rmse in top_2_models:
    print(f"Modelo:  {name}, Promedio RMSE: {rmse}")

Model: Linear Regression, Average RMSE: 1.7039834913849377
Model: Ridge Regression, Average RMSE: 1.7038131972557964
Model: Lasso Regression, Average RMSE: 1.7049191919119284
Model: Decision Tree, Average RMSE: 1.1730418873497936
Model: Random Forest, Average RMSE: 1.2297282328770218
Model: Gradient Boosting, Average RMSE: 1.2958638329746899
Model: Support Vector Regression, Average RMSE: 1.4067877324088645

 Los 2 mejores Modelos son: 
Modelo:  Decision Tree, Promedio RMSE: 1.1730418873497936
Modelo:  Random Forest, Promedio RMSE: 1.2297282328770218


In [89]:

# Parámetros para Decision Tree y Random Forest
param_grid_dt = {
    'regressor__max_depth': [None, 5, 10, 20],
    'regressor__min_samples_split': [2, 5, 10],
    'regressor__min_samples_leaf': [1, 2, 4]
}

param_grid_rf = {
    'regressor__n_estimators': [50, 100, 200],
    'regressor__max_depth': [None, 5, 10],
    'regressor__min_samples_split': [2, 5],
    'regressor__min_samples_leaf': [1, 2]
}


# Haremos GridSearchCV para tener el mejor modelo:
def tune_model(model, param_grid, X, y):
    pipeline = Pipeline([
        ('transformer', ct),
        ('regressor', model)
    ])
    # Changed 'Error 2 negativo' to 'neg_mean_squared_error'
    grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_search.fit(X, y)
    return grid_search.best_estimator_


# Ajustar Decision Tree
best_dt_model = tune_model(DecisionTreeRegressor(), param_grid_dt, X, y)
print("\n Mejor Modelo de Decisión de Árbol:")
print(best_dt_model)

# Ajustar Random Forest
best_rf_model = tune_model(RandomForestRegressor(), param_grid_rf, X, y)
print("\n Mejor modelo de Random Forest:")
print(best_rf_model)

# Evaluar los mdelos usando validación cruzada
def evaluate_best_model(model, X, y):
    # Changed 'Error 2 negativo' to 'neg_mean_squared_error'
    cv_scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
    rmse_scores = np.sqrt(-cv_scores)
    return rmse_scores.mean(), rmse_scores

dt_rmse_avg, dt_rmse_scores = evaluate_best_model(best_dt_model, X, y)
rf_rmse_avg, rf_rmse_scores = evaluate_best_model(best_rf_model, X, y)

print(f"\nDecisión de Árbol - RMSE Promedio: {dt_rmse_avg}, RMSE score: {dt_rmse_scores}")
print(f"Random Forest - RMSE Promedio: {rf_rmse_avg}, RMSE score: {rf_rmse_scores}")

# Seleccionamos el mejor modelo, basado en RMSE, después de ajuste
if dt_rmse_avg < rf_rmse_avg:
    final_model = best_dt_model.named_steps['regressor']
else:
    final_model = best_rf_model.named_steps['regressor']

print(f"\n Modelo Final Seleccionado: {type(final_model).__name__}")



 Mejor Modelo de Decisión de Árbol:
Pipeline(steps=[('transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['Impresiones', 'Posición',
                                                   'CTR']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Consultas principales'])])),
                ('regressor',
                 DecisionTreeRegressor(max_depth=10, min_samples_split=5))])

 Mejor modelo de Random Forest:
Pipeline(steps=[('transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['Impresiones', 'Posición',
                                             

Basándonos en los resultados obtenidos, y considerando que el DecisionTreeRegressor ha sido seleccionado como el mejor modelo después del proceso de ajuste de hiperparámetros y validación cruzada, podemos concluir que este modelo ofrece un mejor rendimiento en la predicción de la variable objetivo (en este caso, 'Clics') en comparación con el RandomForestRegressor para este conjunto de datos específico. Su menor RMSE promedio indica una menor diferencia entre los valores predichos y los valores reales.

